In [1]:
from prody import *
from matplotlib.pylab import *
from sklearn import*
import sklearn
import numpy 
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
from __future__ import division
import os
import subprocess
import copy
import time
import numpy as np
import random as rand
import scipy.sparse
import scipy.sparse.linalg
from scipy.sparse import csr_matrix as sparsify
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import colorsys
import shutil
from Bio.PDB.PDBParser import PDBParser
from Bio import pairwise2
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from scipy.stats import t
from scipy.stats import scoreatpercentile 
from optparse import OptionParser

/home/akp47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/akp47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/akp47/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  

In [2]:
ion()

In [3]:
# using the pickle file from HAN getting the refined sequences

import _pickle as cPickle
SEQ, consensus = cPickle.load(open("SEQ3_PF00028.pkl","rb"))
print (len(consensus))
print(len(SEQ))
msa = numpy.asarray(SEQ)
msa_refine = numpy.chararray((6210,93))
for i in range(6210):
    msa_refine[i] = msa[i]

93
6210


## Required functions 

In [4]:
def pdbSeq(pdbid, chain='B', path2pdb='', calcDist=1):

    ''' Extract sequence, position labels and matrix of distances from a PDB file.

    

    **Arguments:**

       -  `pdbid` = PDB identifier (four letters/numbers)

       -  `chain` = PDB chain identifier

       -  `path2pdb` = location of the PDB file

       -  `calcDist` = calculate a distance matrix between all pairs of positions, default is 1



    :Example:

       >>> sequence, labels, dist = pdbSeq(pdbid, chain='A', path2pdb=path2structures) '''

    # Table of 3-letter to 1-letter code for amino acids

    aatable = {'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C', 'GLN': 'Q',\

     'GLU': 'E', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K', 'MET': 'M',\

     'PHE': 'F', 'PRO': 'P', 'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'}

    # Read PDB structure:

    structure = PDBParser().get_structure(pdbid, path2pdb+pdbid+'.pdb')

    # Fill up sequence and label information

    sequence = ''; labels = list()

    residues = [res for res in structure[0][chain] if res.get_id()[0] == ' ']

    for res in residues:

        labels.append(str(res.get_id()[1])+str(res.get_id()[2]).strip())

        try:

            sequence += aatable[res.get_resname()]

        except:

            sequence += 'X'

    # Distances between residues (minimal distance between atoms, in angstrom): 

    dist = np.zeros((len(residues), len(residues)))

    if (calcDist == 1):

        for n0, res0 in enumerate(residues):

            for n1, res1 in enumerate(residues):

                dist[n0,n1] = min([atom0 - atom1 for atom0 in res0 for atom1 in res1])

        return sequence, labels, dist

    else:

        return sequence, labels
    
def readAlg(filename):
    ''' Read in a multiple sequence alignment in fasta format, and return the 
    headers and sequences.
    >>> headers, sequences = readAlg(filename) '''
    filelines = open(filename, 'r').readlines()
    headers = list(); sequences = list(); notfirst = 0
    for line in filelines:
        if line[0] == '>':
            if notfirst > 0: sequences.append(seq.replace('\n','').upper())
            headers.append(line[1:].replace('\n',''))
            seq = ''; notfirst = 1
        elif line != '\n': seq += line
    sequences.append(seq.replace('\n','').upper())
    return headers, sequences

def makeATS(sequences, refpos, refseq, iref=0, truncate=False):
    ''' If specified, truncate the alignment to the structure (assumes MSAsearch_ has already been run
    to identify the reference sequence (iref)) and produce a mapping (ats) between alignment positions and the positions in the reference sequence (refpos).
    
    .. _MSAsearch: scaTools.html#scaTools.MSAsearch
     **Arguments:**
       -  sequences
       -  reference positions 
       -  reference sequence
       -  iref, the index of the sequence in the alignment with the highest identity to the reference
    :Keyword Arguments:
       truncate  -- truncate the alignment to positions that align to the reference sequence.
    :Example:
      >>> sequences_trun, ats_new = sca.makeATS(sequences_full, ats_pdb, seq_pdb, i_ref)
    '''
    if truncate == True:
        print("truncating to reference sequence...")
        # Removing gaps:
        pos_ref = [i for i,a in enumerate(refseq) if a != '-']
        seq_ref = ''.join([refseq[i] for i in pos_ref])
        ats_ref = [refpos[i] for i in pos_ref]
        pos_alg = [i for i,a in enumerate(sequences[iref]) if a != '-']
        seq_tr = [''.join([sq[i] for i in pos_alg]) for sq in sequences]
        # Positions to keep in the alignment and pbd sequences
        # (no gap in any of them after co-alignment):
        seqal_ref, seqal_alg, _, _, _ = pairwise2.align.globalms(seq_ref, seq_tr[iref],\
                                         2, -1, -.5, -.1)[0]
        keep_ref, keep_alg = list(), list()
        j_ref, j_alg = 0, 0
        for i in range(len(seqal_ref)):
            if seqal_ref[i] != '-' and seqal_alg[i] != '-':
                keep_ref.append(j_ref)
                keep_alg.append(j_alg)
            if seqal_ref[i] != '-': j_ref += 1
            if seqal_alg[i] != '-': j_alg += 1
        sequences_out = [''.join([sq[i] for i in keep_alg]) for sq in seq_tr]
        ats_out = [ats_ref[i] for i in keep_ref]
    else:
        tmp = sequences[iref].replace('-','.')
        refseq = refseq.replace('-','');
        seqal_ref, seqal_alg, _, _, _ = pairwise2.align.globalms(refseq, tmp,\
                                         2, -1, -.5, -.1)[0]
        print ('Len refseq %i, len refpos %i, Len alg seq %i, len pairalg %i, len gloalg %i' % (len(refseq),len(refpos), len(tmp),len(seqal_alg),len(sequences[0])))
        #print seqal_ref
        #print seqal_alg
        ats_out = list()
        j_ref = 0
        j_pdb = 0
        for i in range(len(seqal_alg)):
            if seqal_alg[i] == '.' and seqal_ref[i] == '-':
                ats_out.insert(j_ref,'-')
                j_ref += 1
            elif seqal_alg[i] != '.' and seqal_alg[i] != '-':
                if seqal_ref[i] != '-':
                    ats_out.insert(j_ref,refpos[j_pdb])
                    j_ref += 1
                    j_pdb += 1
                else:
                    ats_out.insert(j_ref,'-')
                    j_ref += 1
            elif seqal_alg[i] == '.' and seqal_ref[i] != '-':
                ats_out.insert(j_ref, refpos[j_pdb])
                j_ref += 1
                j_pdb += 1
            elif seqal_alg[i] == '-':
                j_pdb += 1
        sequences_out = sequences
    return sequences_out, ats_out

import scipy
from scipy.stats import t
from scipy.stats import scoreatpercentile 

def getCutoff(M, p_cut=None):
    
    eigenValues,eigenVectors = np.linalg.eigh(M)
    idx = (-eigenValues).argsort()   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    for k in range(eigenVectors.shape[1]):
        if np.sign(np.mean(eigenVectors[:,k])) != 0:
            eigenVectors[:,k] = np.sign(np.mean(eigenVectors[:,k]))*eigenVectors[:,k]
    evpos = eigenVectors
    evpos = np.around(evpos, decimals=4)
    
    
    Npos = len(evpos)
    cutoff = list()
    scaled_pdf = list()
    all_fits = list()
    
    k = 0;
    
    pd = scipy.stats.lognorm.fit(evpos[:,0])
    
    #pd = t.fit(evpos[:,k])
    
    all_fits.append(pd)
    iqr = scoreatpercentile(evpos[:,k],75) - scoreatpercentile(evpos[:,k],25)
    binwidth=2*iqr*(len(evpos[:,k])**(-0.33))
    nbins=round((max(evpos[:,k])-min(evpos[:,k]))/binwidth)
    
    h_params = np.histogram(evpos[:,k], int(nbins))
    
    x_dist = np.linspace(min(h_params[1]), max(h_params[1]), num=100)
    area_hist=Npos*(h_params[1][2]-h_params[1][1]);
    
    scaled_pdf.append(area_hist*(scipy.stats.lognorm.pdf(x_dist,pd[0],pd[1],pd[2])))
    
    #print(scaled_pdf)
    cd = scipy.stats.lognorm.cdf(x_dist,pd[0],pd[1],pd[2])
    tmp = scaled_pdf[k].argmax()
    if abs(max(evpos[:,k])) > abs(min(evpos[:,k])):
            tail = cd[tmp:len(cd)]
    else:
        cd = 1 - cd
        tail = cd[0:tmp]
    diff = abs(tail - p_cut);
    x_pos = diff.argmin()
    #print(x_pos)
    cutoff.append(x_dist[x_pos+tmp])
    return evpos, cutoff;

def getResidues(evpos, cutoff=None):
    
    ind = [];
    ind = (np.where(evpos[:,0] > cutoff))
    ind = np.asarray(ind)
    ind = ind + 18;
    return ind;

# Gives residues with high coevolution

def getImpoRes(residues):
    imp_res = []
    for i in range (residues.shape[0]):
        imp_res = (np.append(imp_res,(ats[residues[i]])))
    return imp_res;

# Gives Y observed to calculate F1 score

def getAnnotations(impoRes,consensus):
    consensus = consensus
    impoRes= impoRes
    annotations = np.in1d(consensus, impoRes)
    y_observed = annotations.astype(int)
    return y_observed;




## Get Important Residues

In [5]:
seq_pdb, ats_pdb, dist_pdb = pdbSeq('2ee0', chain='A')
headers, sequences = readAlg('PF00028.fasta')
i_ref=1
sequences, ats = makeATS(sequences, ats_pdb, seq_pdb, i_ref, truncate=False)

Len refseq 114, len refpos 114, Len alg seq 93, len pairalg 168, len gloalg 93


### SCA 

In [6]:
SCA_matrix = buildSCAMatrix(msa_refine, turbo=False)
evpos_sca, cutoff_sca = getCutoff(M=SCA_matrix,p_cut=0.80)
cutoff_sca
imp_res_sca = getResidues(evpos=evpos_sca, cutoff=cutoff_sca)
imp_res_sca = imp_res_sca[0]
imp_res_sca

@> SCA matrix was calculated in 0.17s.


array([ 18,  24,  25,  29,  35,  37,  39,  41,  43,  45,  50,  61,  68,
        70,  77,  79,  84,  90,  92,  95,  97, 105])

### MI

In [7]:
mi_matrix = buildMutinfoMatrix(msa_refine)
evpos_mi, cutoff_mi = getCutoff(M=mi_matrix,p_cut=0.80)
cutoff_mi
imp_res_mi = getResidues(evpos=evpos_mi, cutoff=cutoff_mi)
imp_res_mi = imp_res_mi[0]
imp_res_mi

@> Mutual information matrix was calculated in 0.09s.


array([ 21,  26,  33,  42,  43,  46,  47,  55,  58,  60,  93,  97,  99,
       100, 102, 103, 106, 108])

### PSICOV

In [8]:
psicov_matrix = buildPCMatrix(msa_refine)
evpos_psicov, cutoff_psicov = getCutoff(M=psicov_matrix,p_cut=0.80)
cutoff_psicov
imp_res_psicov = getResidues(evpos=evpos_psicov, cutoff=cutoff_psicov)
imp_res_psicov = imp_res_psicov[0]
imp_res_psicov

@> PC matrix was calculated in 8.58s.


array([ 20,  26,  33,  46,  47,  55,  56,  60,  72,  74,  78,  82,  86,
        89,  93, 103])

### OMES

In [9]:
omes_matrix = buildOMESMatrix(msa_refine)
evpos_omes, cutoff_omes = getCutoff(M=omes_matrix,p_cut=0.80)
cutoff_omes
imp_res_omes = getResidues(evpos=evpos_omes, cutoff=cutoff_omes)
imp_res_omes = imp_res_omes[0]
imp_res_omes

@> OMES matrix was calculated in 0.05s.


array([ 26,  32,  33,  34,  35,  41,  43,  46,  60,  72,  92, 101, 102,
       103, 105, 106, 109])

## DCA

In [10]:
di_matrix = buildDirectInfoMatrix(msa_refine)
evpos_di, cutoff_di = getCutoff(M=di_matrix,p_cut=0.80)
cutoff_di
imp_res_di = getResidues(evpos=evpos_di, cutoff=cutoff_di)
imp_res_di = imp_res_di[0]
imp_res_di

@> DI matrix was calculated in 4.60s.


array([ 19,  20,  24,  33,  39,  41,  42,  43,  77,  79,  92,  95,  96,
        97,  98,  99, 100, 102, 103, 104, 106])

## y Observed

### SCA

In [11]:
evpos_sca_cons,cutoff_sca_cons = getCutoff(M=SCA_matrix,p_cut=0.8)
cons_sca = getResidues(evpos=evpos_sca_cons,cutoff=0)
cons_sca = cons_sca[0]
cons_sca

array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110])

In [12]:
y_sca = getAnnotations(imp_res_sca,cons_sca)
y_sca

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0])

### MI

In [13]:
evpos_mi_cons,cutoff_mi_cons = getCutoff(M=SCA_matrix,p_cut=0.80)
cons_mi = getResidues(evpos=evpos_mi_cons,cutoff=0)
cons_mi = cons_mi[0]
cons_mi

array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110])

In [14]:
y_mi = getAnnotations(imp_res_mi,cons_mi)
y_mi

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0])

### PSICOV

In [15]:
evpos_psicov_cons,cutoff_psicov_cons = getCutoff(M=SCA_matrix,p_cut=0.80)
cons_psicov = getResidues(evpos=evpos_psicov_cons,cutoff=0)
cons_psicov = cons_psicov[0]
cons_psicov

array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110])

In [16]:
y_psicov = getAnnotations(imp_res_psicov,cons_psicov)
y_psicov

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0])

### OMES

In [17]:
evpos_omes_cons,cutoff_omes_psicov = getCutoff(M=SCA_matrix,p_cut=0.80)
cons_omes = getResidues(evpos=evpos_omes_cons,cutoff=0)
cons_omes = cons_omes[0]
cons_omes

array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110])

In [18]:
y_omes = getAnnotations(imp_res_omes,cons_omes)
y_omes

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0])

## DCA

In [19]:
evpos_di_cons,cutoff_di_psicov = getCutoff(M=di_matrix,p_cut=0.80)
cons_di = getResidues(evpos=evpos_omes_cons,cutoff=0)
cons_di = cons_di[0]
cons_di

array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110])

In [20]:
y_di = getAnnotations(imp_res_di,cons_di)
y_di

array([0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0])